In [1]:
%matplotlib qt

In [2]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import json
from sklearn import preprocessing
from networkx.readwrite import json_graph

In [3]:
def read_json_file(filename):
    with open(filename,  "r", encoding="utf8") as f:
        js_graph = json.load(f)
    return json_graph.node_link_graph(js_graph, directed=False, multigraph=True), js_graph

G,data = read_json_file("MC3/MC3.json")
G.is_directed(), G.is_multigraph()

(True, True)

In [4]:
G = G.to_undirected()

print(G.is_directed(), G.is_multigraph())

G.number_of_edges(), G.number_of_nodes()

False True


(24038, 52170)

#### To Pandas DF 

In [5]:
def safe_convert(x):
    try:
        return x[0]
    except:
        return ""

In [6]:
df = pd.DataFrame(data["nodes"])

df.id = df.id.apply(lambda x:x[0])
df.country = df.country.apply(lambda x:x[0])
df.product_services = df.product_services.apply(safe_convert)
df.revenue_omu = df.revenue_omu.apply(safe_convert)
df.type = df.type.apply(safe_convert)

nodes = df.id.values.tolist()

len(nodes), len(set(nodes))#counting without duplicate

(27622, 22929)

In [7]:
df

,country,id,product_services,revenue_omu,type
0,ZH,Jones LLC,Automobiles,310612303.447,Company
1,ZH,"Coleman, Hall and Lopez","Passenger cars, trucks, vans, and buses",162734683.9969,Company
2,Oceanus,Aqua Advancements Sashimi SE Express,Holding firm whose subsidiaries are engaged in...,115004666.6728,Company
3,Utoporiana,Makumba Ltd. Liability Co,"Car service, car parts and accessories, automo...",90986412.5191,Company
4,ZH,"Taylor, Taylor and Farrell",Fully electric vehicles (EVs) and electric veh...,81466666.6728,Company
...,...,...,...,...,...
27617,ZH,"Johnson, Lee and Rodriguez",,,Company Contacts
27618,ZH,"Bowman, Rollins and Griffin",,,Company Contacts
27619,ZH,Hardin Group,,,Company Contacts
27620,ZH,"Crane, Joyce and Jennings",,,Company Contacts


In [8]:
##delete duplicate nodes according to all columns
#df_2 = df.drop_duplicates(subset=["country", "id", "product_services", "revenue_omu", "type"])

In [9]:
df1 = pd.DataFrame(data["links"])

df1.source = df1.source.apply(lambda x: x[0])
df1.target = df1.target.apply(lambda x: x[0])
df1.type = df1.type.apply(lambda x: x[0])

##all nodes from source and target
nodes_edges = df1.source.values.tolist() + df1.target.values.tolist()

len(nodes_edges), len(set(nodes_edges))#counting without duplicate

(48076, 34062)

In [10]:
df1

,source,target,type
0,Lake Chad Catchers Limited Liability Company ...,Erin Flores,Beneficial Owner
1,Lake Chad Catchers Limited Liability Company ...,Linda Lee,Beneficial Owner
2,Lake Chad Catchers Limited Liability Company ...,Sharon Coleman,Beneficial Owner
3,Lake Chad Catchers Limited Liability Company ...,John Rivera,Beneficial Owner
4,Lake Chad Catchers Limited Liability Company ...,Stephen Carpenter,Beneficial Owner
...,...,...,...
24033,Martinez-Diaz,Mark Thomas,Company Contacts
24034,Garcia Inc,Christopher Wagner,Company Contacts
24035,"Potter, Williams and Peters",Mr. Troy Shields,Company Contacts
24036,Tucker-Willis,Robert James,Company Contacts


#### Analysis of Connected Components

In [11]:
%matplotlib qt
#para ver aca

In [14]:
import nxviz as nv

In [15]:
nx.number_connected_components(G)

28176

In [16]:
connected_components = [c for c in sorted(nx.connected_components(G), key=len, reverse=True)]

In [17]:
S = [G.subgraph(c).copy() for c in sorted(nx.connected_components(G), key=len, reverse=True)] ##subgraphs for each component

In [18]:
fig, ax = plt.subplots()
ax = nv.circos(
    S[1] 
)

In [30]:
for v in S[8].edges.data():
    print(v)

(('Judith Anderson',), ("Surf's Up s OJSC Consultants",), {'type': ['Beneficial Owner']})
(('Susan Reed',), ("Gulf of Guinea   & Son's",), {'type': ['Beneficial Owner']})
(('Zoe Marsh',), ('Zambezi Escarpment  Incorporated United',), {'type': ['Beneficial Owner']})
(('Jonathan Carter',), ('Playa de Arena Jetsam AS Holdings',), {'type': ['Beneficial Owner']})
(('Jonathan Carter',), ('Assam  Market Kga Marine ecology',), {'type': ['Beneficial Owner']})
(('Randy Johnson',), ('Haryana Sea  Marine ecology', 'Haryana Sea  Marine ecology'), {'type': ['Company Contacts']})
(('Randy Johnson',), ('Reyes, Jackson and Johnson',), {'type': ['Beneficial Owner']})
(('Ashley Kirby',), ('Ob River AS Consulting',), {'type': ['Company Contacts']})
(('Jacqueline Soto',), ('Ob River AS Consulting',), {'type': ['Beneficial Owner']})
(('Tyler Castro',), ("Gulf of Guinea   & Son's",), {'type': ['Beneficial Owner']})
(('Mizoram  Ltd. Liability Co',), ('Donna Davis',), {'type': ['Company Contacts']})
(('Donna D

In [47]:
nx.draw_spring(S[8], with_labels=True)

#### Add types for source and target fields

In [15]:
df1["source_type"] = df1.source.apply(lambda x: df[df.id==x].type.unique())

In [17]:
df1["target_type"] = df1.target.apply(lambda x: df[df.id==x].type.unique())

In [20]:
columns = ["source", "source_type", "target", "target_type", "type"]

In [22]:
df1 = df1[columns]

In [60]:
##NINGUN TARGET DE LOS LINKS ESTA DEFINIDO EN LA SECCION NODOS

for v in df1.target.values:
    if v in df.id.values:
        print(v)